In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# part 2 meta added

def add_policy(chart_data: pd. DataFrame):
    folder = "/Users/sara/emulator/data/"

    # Metadata have been edited correcting the Sustainable IMP which was wrongly assinged to SSP2
    file = "AR6_meta.csv"

    file = os.path.join(folder, file)
    metadata = pd.read_csv(file, encoding='unicode_escape')
    policy= metadata["Policy_category"]
    policy = policy.fillna("Unknown")

    old_policy = list(policy.unique())
 
    new_policy = ["P2", "P3", "P1", "P1", "P1", "P1", 
                 "P2", "P2", "P0", "P3", "P2", "P1", "P3",  
                 "P0", "P0", "Unknown", "P0", "P0", "P0", "P4" ]

    simple_policy = dict(zip(old_policy, new_policy))


    policy = list(policy.fillna("Unknown"))
    scens = list(metadata["Scenario"])

    not_scens = [sc for sc in list(set(chart_data.Scenario)) if sc not in scens]
    not_scenspol = ["Unknown" for i in range(len(not_scens))]

    scens = scens + not_scens
    policy = policy + not_scenspol

    dict_policy = dict(zip(scens,policy))

    chart_data["Policy"] = chart_data["Scenario"].apply(lambda x: dict_policy[x])
    chart_data["Policy"] = chart_data["Policy"].apply(lambda x: simple_policy[x])    
    return chart_data


def add_category(chart_data: pd. DataFrame):
    folder = "/Users/sara/emulator/data/"

    # Metadata have been edited correcting the Sustainable IMP which was wrongly assinged to SSP2
    file = "AR6_meta.csv"

    file = os.path.join(folder, file)
    metadata = pd.read_csv(file, encoding='unicode_escape')
    policy= metadata["Category_subset"]
    policy = list(policy.fillna("Unknown"))
    scens = list(metadata["Scenario"])

    not_scens = [sc for sc in list(set(chart_data.Scenario)) if sc not in scens]
    not_scenspol = ["Unknown" for i in range(len(not_scens))]

    scens = scens + not_scens
    policy = policy + not_scenspol

    dict_policy = dict(zip(scens,policy))


    chart_data["Category"] = chart_data["Scenario"].apply(lambda x: dict_policy[x])
    category = list(chart_data.Category.unique())
    new_category = ["C1", "C5", "C3", "C4", "C7", "C8", "C6", "C3", "C1", "C2", "no climate asessment", "Unknown"]
    dict_category = dict(zip(category, new_category))
    chart_data["Category"] = chart_data["Category"].apply(lambda x: dict_category[x])

    return chart_data
    


array(['P2a', 'P3b', 'P1c', 'P1a', 'P1b', 'P1d', 'P2', 'P2c', 'P0_1d',
       'P0', 'P3c', 'P2b', 'P1', 'P3a', 'Unknown', 'P0_2', 'P0_1a', 'P4'],
      dtype=object)

In [73]:
baseyear = 2010
endyear = 2100
years = np.arange(baseyear, endyear, step=10)
syears = [str(y) for y in years]

file = "/Users/sara/emulator/data/AR6_Scenarios_Database_World_v1.1.csv"

strings = ["Model",	"Scenario",	"Region",	"Variable",	"Unit"]

selected_var = [
                "Emissions|CO2",
                "Emissions|CO2|Energy and Industrial Processes",
                "Emissions|CO2|Energy|Demand",
                "Emissions|CO2|Energy|Demand|Industry",
                "Emissions|CO2|Energy|Demand|Residential and Commercial",
                "Emissions|CO2|Energy|Demand|Transportation",
                "Emissions|CO2|Energy|Supply",
                "Emissions|CO2|Energy|Supply|Electricity",
                "Emissions|CO2|Energy",
                "Emissions|CO2|Energy|Supply|Gases",
                "Emissions|CO2|Energy|Supply|Liquids",
                "Emissions|CO2|Energy|Supply|Solids",
                ]

selected_col = strings + syears
data = pd.read_csv(file)[selected_col]

my_data = pd.DataFrame(data[data.Variable.isin(selected_var)])

# select variable
selection = "Emissions|CO2"
chart_data = pd.DataFrame(my_data.loc[my_data.Variable==selection])
new_cols = ["C" + str(c) for c in range(len(chart_data))]
chart_data["String"] = new_cols

chart_data = add_policy(chart_data)
chart_data = add_category(chart_data)
selected_col = ["String"] + strings + ["Policy"] + ["Category"] + syears
chart_data = chart_data[selected_col]

chart_data = chart_data.rename(columns=dict(zip(syears, years)))

chart_data = chart_data.drop(["Model", "Region"], axis=1)



In [74]:
chart_data.head()

,String,Scenario,Variable,Unit,Policy,Category,2010,2020,2030,2040,2050,2060,2070,2080,2090
77,C0,ADVANCE_2020_1.5C-2100,Emissions|CO2,Mt CO2/yr,P2,C1,38148.9465,43907.5503,21076.1268,5447.7891,-413.2438,-5258.9563,-7657.0574,-8418.9858,-9964.3564
268,C1,ADVANCE_2020_Med2C,Emissions|CO2,Mt CO2/yr,P2,C5,38148.9465,43943.2767,23872.3830,21409.4609,18343.4203,10980.7007,12896.3541,13166.0372,10392.4725
459,C2,ADVANCE_2020_WB2C,Emissions|CO2,Mt CO2/yr,P2,C3,38148.9465,43943.2767,23879.8703,12899.0741,8213.4431,858.7188,919.0067,1595.5126,-915.7661
650,C3,ADVANCE_2030_Med2C,Emissions|CO2,Mt CO2/yr,P3,C5,38149.3009,43970.3298,41558.2299,17620.4085,13654.1067,8049.9381,9749.0081,10536.1824,7374.9209
841,C4,ADVANCE_2030_Price1.5C,Emissions|CO2,Mt CO2/yr,P3,C4,38102.4430,44557.4780,41746.2485,8994.4760,87.1537,-4686.0844,-6589.1981,-6495.7647,-7435.7386


In [84]:
# estimate sum over columns
chart_data['Sum']=chart_data[years].sum(axis=1)

allmin = pd.DataFrame()
allmax = pd.DataFrame()
for category in chart_data.Category.unique():
    data = pd.DataFrame(chart_data.loc[chart_data.Category == category])
    data = data.sort_values(by="Sum", ascending=True)
    allmin = pd.concat((allmin,pd.DataFrame(data.iloc[0]).transpose()), axis=0)
    data = data.sort_values(by="Sum", ascending=False)
    allmax = pd.concat((allmax,pd.DataFrame(data.iloc[0]).transpose()),axis=0)



,String,Scenario,Variable,Unit,Policy,Category,2010,2020,2030,2040,2050,2060,2070,2080,2090,Sum
158348,C680,EMF33_tax_lo_none,Emissions|CO2,Mt CO2/yr,P2,C5,29.96,30.48,11.84,4.678,8.329,11.45,22.23,29.19,34.68,182.837
158048,C679,EMF33_tax_hi_none,Emissions|CO2,Mt CO2/yr,P2,C4,29.96,30.48,12.06,8.001,7.547,12.0,27.8,39.89,33.51,201.248
107983,C452,R_MAC_30_n8,Emissions|CO2,Mt CO2/yr,P2,C1,36026.15982,39923.678886,-7999.992267,-7999.998335,-8000.000287,-8000.00442,-7999.999816,-8000.00445,-8000.010057,19949.829075
98748,C411,EMF33_WB2C_limbio,Emissions|CO2,Mt CO2/yr,P2,C3,35793.127826,4143.019359,-2329.455178,-416.832065,603.031271,907.433905,-383.103128,-2563.740996,-4663.189888,31090.291105
227486,C832,BPS,Emissions|CO2,Mt CO2/yr,P2,no climate asessment,NaN,27754.0,14483.0,6768.0,0.0,NaN,NaN,NaN,NaN,49005.0
412925,C1186,EMF33_1.5C_cost100,Emissions|CO2,Mt CO2/yr,P2,C2,34045.15625,38511.03125,10375.259766,5185.838379,513.442383,-4844.283203,-8202.635742,-9723.092773,-11018.397461,54842.318848
104994,C444,NGFS2_Below 2°C,Emissions|CO2,Mt CO2/yr,Unknown,Unknown,34654.23876,41741.75309,32406.95474,18922.58369,8278.475528,1076.803384,-2832.526163,-2711.414102,-2180.161602,129356.707325
122415,C529,CD-LINKS_INDCi,Emissions|CO2,Mt CO2/yr,P1,C7,30080.149547,36318.739374,35609.360777,32749.662642,29928.948185,NaN,NaN,NaN,NaN,164686.860524
457590,C1293,CO_NDCplus,Emissions|CO2,Mt CO2/yr,P1,C6,32079.461819,34842.003966,35178.476829,34786.257191,35453.838618,NaN,NaN,NaN,NaN,172340.038423
88096,C337,EMF30_D_Frozen-CH4,Emissions|CO2,Mt CO2/yr,P0,C8,37407.34635,42744.225336,48171.137015,53196.540265,60503.974839,72227.458722,NaN,NaN,NaN,314250.682525


In [82]:
pd.DataFrame(data.iloc[0]).transpose()

,String,Scenario,Variable,Unit,Policy,Category,2010,2020,2030,2040,2050,2060,2070,2080,2090,Sum
506631,C1414,SSP5-Baseline,Emissions|CO2,Mt CO2/yr,P1,C8,36371.1246,44610.3893,56726.452,69861.617,84436.4661,101301.6164,117499.8259,129499.3478,130397.5318,770704.3709


In [13]:

# part 3 transpose
chart_data1 = chart_data.transpose()
chart_data1.columns = chart_data1.loc[chart_data1.index=="String"].values[0]
chart_data1 = chart_data1.iloc[3:,:]
print(chart_data, chart_data1)

       String                Scenario       Variable       Unit Policy  \
77         C0  ADVANCE_2020_1.5C-2100  Emissions|CO2  Mt CO2/yr    P2a   
268        C1      ADVANCE_2020_Med2C  Emissions|CO2  Mt CO2/yr    P2a   
459        C2       ADVANCE_2020_WB2C  Emissions|CO2  Mt CO2/yr    P2a   
650        C3      ADVANCE_2030_Med2C  Emissions|CO2  Mt CO2/yr    P3b   
841        C4  ADVANCE_2030_Price1.5C  Emissions|CO2  Mt CO2/yr    P3b   
...       ...                     ...            ...        ...    ...   
692567  C1866            CD-LINKS_NPi  Emissions|CO2  Mt CO2/yr    P1b   
692735  C1867   CD-LINKS_NPi2020_1000  Emissions|CO2  Mt CO2/yr    P2a   
692903  C1868   CD-LINKS_NPi2020_1600  Emissions|CO2  Mt CO2/yr    P2a   
693051  C1869    CD-LINKS_NPi2020_400  Emissions|CO2  Mt CO2/yr    P2a   
693219  C1870       CD-LINKS_NoPolicy  Emissions|CO2  Mt CO2/yr    P1a   

                2010          2020          2030          2040          2050  \
77      38148.946500  43907.550

In [6]:


cols = ["C" + str(c) for c in range(data.values.shape[1])]
data.columns = cols
data.transpose()


,0,1,2,3,4,5,6,7,8,9,...,693289,693290,693291,693292,693293,693294,693295,693296,693297,693298
C0,AIM/CGE 2.0,AIM/CGE 2.0,AIM/CGE 2.0,AIM/CGE 2.0,AIM/CGE 2.0,AIM/CGE 2.0,AIM/CGE 2.0,AIM/CGE 2.0,AIM/CGE 2.0,AIM/CGE 2.0,...,WITCH-GLOBIOM 4.4,WITCH-GLOBIOM 4.4,WITCH-GLOBIOM 4.4,WITCH-GLOBIOM 4.4,WITCH-GLOBIOM 4.4,WITCH-GLOBIOM 4.4,WITCH-GLOBIOM 4.4,WITCH-GLOBIOM 4.4,WITCH-GLOBIOM 4.4,WITCH-GLOBIOM 4.4
C1,ADVANCE_2020_1.5C-2100,ADVANCE_2020_1.5C-2100,ADVANCE_2020_1.5C-2100,ADVANCE_2020_1.5C-2100,ADVANCE_2020_1.5C-2100,ADVANCE_2020_1.5C-2100,ADVANCE_2020_1.5C-2100,ADVANCE_2020_1.5C-2100,ADVANCE_2020_1.5C-2100,ADVANCE_2020_1.5C-2100,...,CD-LINKS_NoPolicy,CD-LINKS_NoPolicy,CD-LINKS_NoPolicy,CD-LINKS_NoPolicy,CD-LINKS_NoPolicy,CD-LINKS_NoPolicy,CD-LINKS_NoPolicy,CD-LINKS_NoPolicy,CD-LINKS_NoPolicy,CD-LINKS_NoPolicy
C2,World,World,World,World,World,World,World,World,World,World,...,World,World,World,World,World,World,World,World,World,World
C3,AR6 climate diagnostics|Effective Radiative Fo...,AR6 climate diagnostics|Effective Radiative Fo...,AR6 climate diagnostics|Effective Radiative Fo...,AR6 climate diagnostics|Effective Radiative Fo...,AR6 climate diagnostics|Effective Radiative Fo...,AR6 climate diagnostics|Effective Radiative Fo...,AR6 climate diagnostics|Effective Radiative Fo...,AR6 climate diagnostics|Effective Radiative Fo...,AR6 climate diagnostics|Effective Radiative Fo...,AR6 climate diagnostics|Effective Radiative Fo...,...,Primary Energy|Wind,Primary Energy|Biomass|Modern|w/ CCS,Capacity|Electricity,Capacity|Electricity|Oil,Food Demand,Final Energy|Geothermal,Final Energy|Hydrogen,Final Energy|Solar,Capacity|Electricity|Solar|CSP,Capacity|Electricity|Wind|Offshore
C4,W/m^2,W/m^2,W/m^2,W/m^2,W/m^2,W/m^2,W/m^2,W/m^2,W/m^2,W/m^2,...,EJ/yr,EJ/yr,GW,GW,kcal/cap/day,EJ/yr,EJ/yr,EJ/yr,GW,GW
C5,2.206147,1.829122,2.346395,2.495031,2.90358,2.157178,1.80932,2.282659,2.417368,2.86593,...,1.317832,0.000175,4751.230241,368.812242,2805.710512,0.0,0.0,0.0,3.69106,3.468477
C6,2.739332,2.373218,2.87615,3.026901,3.396955,2.638732,2.298435,2.759419,2.894698,3.332385,...,4.709522,0.000085,6417.106916,363.261728,2904.513321,0.0,0.0,0.0,18.030343,6.523548
C7,2.830737,2.504397,2.950698,3.072229,3.401024,2.920518,2.593227,3.019699,3.136544,3.477681,...,9.895566,0.000137,8878.861025,335.255607,3000.486769,0.0,0.0,0.0,53.539354,4.744129
C8,2.716747,2.406405,2.827834,2.940928,3.248263,2.808793,2.526722,2.921064,3.021157,3.312953,...,16.932968,0.000127,11178.318734,325.369549,3073.07089,0.0,0.0,0.0,117.64485,3.543692
C9,2.604978,2.340373,2.697309,2.792838,3.066408,2.66529,2.420084,2.759433,2.869644,3.116089,...,27.274696,0.000178,12728.35139,324.435151,3141.826611,0.0,0.0,0.0,212.824859,2.73412
